In [20]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import tensorflow as tf
import csv
import pandas as pd
import model as md
import utils as ut

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# configurations
# inFilePath = "/asic/projects/C/CMS_PIX_28/pixelAV_datasets/unshuffled_DO_NOT_DELETE/initial_studies/dataset14/unflipped"
# inFilePath = "/asic/projects/C/CMS_PIX_28/pixelAV_datasets/unshuffled_DO_NOT_DELETE/dataset_2sNoise/dataset_2sNoise_50x12P5_16x16_100e-sigma_parquets/unflipped"
inFilePath = "/asic/projects/C/CMS_PIX_28/pixelAV_datasets/unshuffled_DO_NOT_DELETE/dataset_2s16x16/dataset_2s_16x16_50x12P5_parquets/unflipped"
outDir="./tmp_16x6"
confs = [
    {"qm_charge_levels" : [400, 1600, 2400], "qm_quant_values" : [0, 1, 2, 3], "zero_pad": False},
#     {"qm_charge_levels" : [500, 1600, 2400], "qm_quant_values" : [0, 1, 2, 3]},
#     {"qm_charge_levels" : [600, 1600, 2400], "qm_quant_values" : [0, 1, 2, 3]},
#     {"qm_charge_levels" : [700, 1600, 2400], "qm_quant_values" : [0, 1, 2, 3]},
#     {"qm_charge_levels" : [800, 1600, 2400], "qm_quant_values" : [0, 1, 2, 3]},
#     {"qm_charge_levels" : [900, 1600, 2400], "qm_quant_values" : [0, 1, 2, 3]},
#     {"qm_charge_levels" : [1000, 1600, 2400], "qm_quant_values" : [0, 1, 2, 3]},
#     {"qm_charge_levels" : [1100, 1600, 2400], "qm_quant_values" : [0, 1, 2, 3]},
#     {"qm_charge_levels" : [1200, 1600, 2400], "qm_quant_values" : [0, 1, 2, 3]},
#     {"qm_charge_levels" : [1300, 1600, 2400], "qm_quant_values" : [0, 1, 2, 3]},
#     {"qm_charge_levels" : [1600, 2000, 2400], "qm_quant_values" : [0, 1, 2, 3]},
]



In [22]:
# load the parquet data, filter on y-profile bin, and quantize (all done in loadParquetData)
for conf in confs:
    if not os.path.exists(outDir):
        os.makedirs(outDir)
    print(f"Running with configuration: {conf}")
    conf["qm"] = ut.loadParquetData(inFilePath=inFilePath, qm_charge_levels = conf["qm_charge_levels"], qm_quant_values = conf["qm_quant_values"], zero_pad = conf["zero_pad"],outDir=outDir)

Running with configuration: {'qm_charge_levels': [400, 1600, 2400], 'qm_quant_values': [0, 1, 2, 3], 'zero_pad': False}


100%|██████████| 82/82 [00:19<00:00,  4.22it/s]


(169614,) (169614,) (169614,)
169614
Creating yprofiles


In [23]:
# Save asic compout
outDict = conf["qm"]
y_local_bins = np.linspace(-8.1, 8.1, 13)
bin_number = 6
y_local_min, y_local_max = y_local_bins[bin_number], y_local_bins[bin_number + 1]
# create compout of y-local subset
if conf["qm"]["outDir"] is not None:
    compout_file_name = os.path.join(conf["qm"]["outDir"], f'compouts_ylocal_{y_local_min:.2f}_{y_local_max:.2f}.csv')
    ut.yprofileToCompoutWrite(conf["qm"]["yprofiles"], compout_file_name)
    outDict["compout_file_name"] = compout_file_name

for conf in confs:
    conf["filtered_qm"] = conf["qm"]

Making compout of y-local subset
   done!


In [24]:
# create model
shape = 16 # y-profile ... why is this 16 and not 8?
nb_classes = 3 # positive low pt, negative low pt, high pt
first_dense = 58 # shape of first dense layer

# keras model
model_file = "/fasic_home/gdg/research/projects/CMS_PIX_28/directional-pixel-detectors/multiclassifier/models/ds8l6_padded_noscaling_keras_d58model.h5"
model = md.CreateModel(shape, nb_classes, first_dense, model_file = model_file)

# qkeras model
# qmodel_file = "/fasic_home/gdg/research/projects/CMS_PIX_28/directional-pixel-detectors/multiclassifier/models/ds8l6_padded_noscaling_qkeras_foldbatchnorm_d58w4a8model.h5"
qmodel_file = "./16x16quantized/model.h5"
qmodel = md.CreateQModel(shape, model_file=qmodel_file)
# generate hls model
ut.gen_hls_model(qmodel, output_dir=outDir)
# prepare weights for ASIC
ut.prepareWeights(os.path.join(outDir, "firmware/weights/"))

Model: "model_912"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 16)]              0         
                                                                 
 dense1 (Dense)              (None, 58)                986       
                                                                 
 batch_normalization_456 (B  (None, 58)                232       
 atchNormalization)                                              
                                                                 
 relu1 (Activation)          (None, 58)                0         
                                                                 
 dense2 (Dense)              (None, 3)                 177       
                                                                 
 linear (Activation)         (None, 3)                 0         
                                                         

/asic/projects/C/CMS_PIX_28/testing/tools/venvs/p3.11.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Done


'./tmp_16x6/firmware/weights/b5_w5_b2_w2_pixel_bin.csv'

In [25]:
# Compute loss and accuracy manually
def getLA(y, predictions, loss_fn, acc_metric=tf.keras.metrics.SparseCategoricalAccuracy()):
    loss = loss_fn(y, predictions).numpy()
    acc_metric.update_state(y, predictions)
    accuracy = acc_metric.result().numpy()
    return loss, accuracy

# evaluating
verbose = 1
batch_size = 2048

# loop over the confs
for conf in confs:
    
    # make predictions
    for m, name in zip([model, qmodel], ["keras", "qkeras"]):
        print(f"Evaluating {name} model...")
        conf[f"{name}_predictions"] = m.predict(conf["qm"]["yprofiles"], batch_size = batch_size, verbose=verbose)
        # predictions = np.argmax(predictions, axis=1)
        predFileName = os.path.join(conf["qm"]["outDir"], f"{name}_predictions.npy")
        np.save(predFileName, conf[f"{name}_predictions"])
        model_loss, model_acc = getLA(conf["qm"]["clslabels"], conf[f"{name}_predictions"], md.custom_loss_function)
        print(f"Finished evaluating {name} model with loss: {model_loss}, accuracy: {model_acc}, predictions saved to {predFileName}")
        print()


Evaluating keras model...
83/83 [==============================] - 0s 1ms/step
Finished evaluating keras model with loss: 1.2632927894592285, accuracy: 0.6009055972099304, predictions saved to ./tmp_16x6/400_1600_2400/keras_predictions.npy

Evaluating qkeras model...
83/83 [==============================] - 0s 2ms/step
Finished evaluating qkeras model with loss: 0.6724108457565308, accuracy: 0.6333910822868347, predictions saved to ./tmp_16x6/400_1600_2400/qkeras_predictions.npy

